# HDFS

scalable File System that handles the failure of nodes without data and can scale up horizontally to any number of nodes. The initial goal of HDFS was to serve large data files with high read and write performance.

- **Fault tolerance**
- **Streaming data access**: HDFS works on a write once read many principle. HDFS does not wait for the entire file to be read before sending data to the client; instead, it sends data as soon as it reads it. The client can immediately process the received stream, which makes data processing efficient.
- **Scalability**: Storing a huge number of small files is generally not recommended; the size of the file should be equal to or greater than the block size. Small files consume memory from name node.
- **Simplicity**
- **High availability**


## HDFS Architecture

- Slow porcessing of large datasets on one computer
    - Isolate compute and data
    - Simultaneous processing on multiple chunks of data
- Movement of large datasets between data nodes
    - data replication on multiple nodes
    - mode compute closer to data node
- Mutliple access randomly by many users modifying files mights cause inconsistencies
    - Only append and truncation allowed

![](./images/hdfsarchitecture.png)

Architecture:
- Data Plane:
    - Data blocks
    - Replication
    - Checkpoints
    - File metadata
- Control Plane: 
    - NameNodes
    - DataNodes
    - JournalNodes
    - Zookeeper

**Control Plane**  
**NameNode**  
- All data operations will first go through a NameNode and then to other relevant Hadoop components. 
- The NameNode manages the File System namespace. 
- It stores the File System tree and metadata of files and directories namely _File System namespace_, image (_fsimage_) files, and _edit logs_ files.

**DataNode**  
- They perform data block operations (creation, modification, or deletion) based on instructions that are received from NameNodes or HDFS clients. 
- They host data processing jobs such as MapReduce. 
- They report back block information to NameNodes.
- DataNodes also communicate between each other in the case of data replication.

**JournalNode**  
- With NameNode high availability, there was a need to manage edit logs and HDFS metadata between a active and standby NameNodes. 
- JournalNodes were introduced to efficiently share edit logs and metadata between two NameNodes.
- JournalNodes exercise concurrency write locks to ensure that edit logs are written by one active NameNode at a time.

**Zookeeper**  
- HA without automatic failover would have manual intervention to bring NameNode services back up in the event of failure. 
- Zookeeper Quorum and Zookeeper Failover controller, also known as ZKFailoverController (ZKFC). 
- Zookeeper maintains data about NameNode health and connectivity. 
- It monitors clients and notifies other clients in the event of failure. 
- Zookeeper maintains an active persistent session with each of the NameNodes, and this session is renewed by each of them upon expiry. 
- In the event of a failure or crash, the expired session is not renewed by the failed NameNode.

**Data Plane**  
**Replication**  
**Chunking**

## HDFS Communication

![](./images/hdfscommarch.png)

## Metadata Management

NameNode keeps the complete _fsimage_ in memory so that all the metadata information requests can be served in the smallest amount of time possible and persist fsimage and edit logs on the disk.

# do some hdfs operations
`hdfs dfs -mkdir /testdir`  
`hdfs dfs -touch /testdir/somefile`  
`hdfs dfs -copyFromLocal /etc/hosts /testdir/`  
`hdfs dfs -cat /testdir/hosts`  

**Web UI**  
`$ hdfs dfs -chown -R o+rwx /`

to fetch the fsimage in readable format  
`hdfs dfsadmin -fetchImage /opt/hadoop`  

to read the file using Offline Image Viewer tool  
`hdfs oiv -i /opt/hadoop/fsimage_0000000000000000000 -o /opt/hadoop/fsimage_output.csv -p Delimited`

to fetch edits file  
`hdfs oev -i /tmp/hadoop-hadoop/dfs/name/current/edits_inprogress_0000000000000000001 -o /tmp/edits-log.xml`  


# YARN

## Before YARN

**MapReduce V1**
* Single JobTracker process (One per cluster)
    * Manages MapReduce jobs
    * Distributes tasks to JobTrackers
* Multiple TaskTracker processes (one per slave node)
    * Starts and monitors MapReduce tasks

**Limitations**
* Resource allocation:
    * Slave nodes are configure with a fixed number of "slots" to run Map and Reduce tasks
* Resource management process limitation:
    * One JobTracker per cluster
* Job types:
    * Limited to MapReduce jobs only

## Enter YARN

* YARN supports multiple distributed processing framework:
    * MapReduce V2
    * Impala
    * Spark
    * Etc..

**YARN Daemons**
* Resource Manager - one per cluster
    * Controls application startup
    * Schedules resources on the slave nodes
* Node Manager - one per slave node
    * Starts all processes for a running application
    * Manages resources on the slave nodes
* Job History Server - one per cluster
    * Archival of job log files


![](./images/yarn_architecture.gif)

**YARN Architecture**
* Containners
    * Allocation of Containers is done by the ResourceManager process
    * Containers allocate CPU and memory on a slave node
    * Containers run an application task
* Application Masters
    * Single Application Master per application
    * Runs inside a Container
    * Responsible for requesting additional containers on behalf of the application itself
* Task
    * A single user-submitted job. An applcation is broken down into multiple tasks
    * Each task runs in a container in a Hadoop slave node


**YARN Logs**
* Multiple processes running on different nodes
* YARN aggregates local logs from NodeManagers to central location - HDFS
* Logs are ordered by application / job
* Accessible on HDFS or using web UI
* Disabled by default, usually enabled after installation

**Job execution with the current setup**
In a Hadoop YARN environment, the job execution lifecycle involves several components, each playing specific roles in managing the distributed processing of data. Given your cluster setup, here's a detailed step-by-step explanation of the job execution lifecycle:

### Cluster Setup

- **Node 1**: NameNode
- **Node 2**: DataNode only
- **Node 3**: ResourceManager only
- **Node 4**: NodeManager only

### Job Execution Lifecycle

1. **Job Submission**:
   - A user or client submits a job to the YARN ResourceManager. This submission typically includes the application's JAR file, necessary resource requests (e.g., memory, CPU requirements), and configuration parameters.

2. **ResourceManager Role**:
   - The ResourceManager, running on Node 3, is responsible for resource allocation across the cluster. It forms the cornerstone of the job's lifecycle and resource scheduling.

3. **ApplicationMaster Initialization**:
   - Upon receiving a job submission, the ResourceManager negotiates resources on behalf of the job and initializes an ApplicationMaster (AM) to manage the job's execution. The ResourceManager allocates a container on Node 4 (running NodeManager) for the ApplicationMaster since it's the only node equipped to execute tasks.

4. **ApplicationMaster Responsibilities**:
   - The ApplicationMaster coordinates the execution of tasks. It requests resources (containers) from the ResourceManager based on job needs and schedules task executions. 
   - For a MapReduce job, the AM will handle splitting the job into map tasks that process data blocks and reduce tasks for aggregation.

5. **Container Allocation and Execution**:
   - The NodeManager (on Node 4) is responsible for managing the execution of tasks within the allocated containers. 
   - Task executors download the necessary code and dependencies from HDFS and execute the map/reduce tasks.
   - Since your setup has only one DataNode (on Node 2) and one NodeManager (on Node 4), map tasks will read data remotely from Node 2, resulting in additional data transfer over the network, which could be a performance constraint.

6. **DataNode Role**:
   - Running only on Node 2, the DataNode stores the data blocks in HDFS. Although it doesn't execute tasks, it plays a critical role in providing data blocks to the map tasks initiated by the NodeManager on Node 4.

7. **Task Completion and Reporting**:
   - Once tasks are completed, the NodeManager reports success or failure of individual task attempts to the ApplicationMaster. 
   - Any failed tasks may be restarted, subject to job configuration, until they succeed or their attempts run out.

8. **Job Completion and Cleanup**:
   - Upon successful task completion and subsequent result aggregation, the ApplicationMaster conveys the job's status and final results back to the ResourceManager.
   - The ResourceManager acknowledges the completion, and the ApplicationMaster releases its resources, wrapping up the job's lifecycle.
   - Log aggregation (if enabled) will consolidate logs from the NodeManager to a central location on HDFS for easier access and debugging.

9. **Result Access**:
   - The results of the job are typically written back to HDFS, accessible by users or client applications for further use. The NameNode (on Node 1) manages metadata for this data, guiding future access.


**Exmaple**  

* Create HDFS dir for logs:  
    * `$ hdfs dfs -mkdir /app-logs`
    * `$ hdfs dfs -chown -R 1777 /app-logs`  
* Run a job:  
    * From the name node run `$ hadoop jar /opt/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.6.jar pi 4 10000`
    * Switch to the YARN ResourceManager WebUI and examine the App status
    * Switch to the name node Web UI to examine the logs 
* Check using CLI
    * `$ yarn application -list -appStates ALL`
    * `$ yarn logs -applicationId <application-id>`